In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv(r"D:\个人文件\Lincoln University\COMP647 Machine Learning\supply_chain_deliveries.csv")
df.sample(5)

,WorkDate,Customer,Location,BusinessType,OrderCount,NumberOfPieces,TotalRevenue
122324,2025/4/30,Home Goods,Sacramento,Middle Mile,35,175.0,740.82
84924,2023/9/10,Shein,Dallas,Middle Mile,12,48.0,1004.28
24905,2021/1/31,Home Goods,Los Angeles,Middle Mile,13,65.0,658.24
43473,2021/11/20,Target,Los Angeles,Middle Mile,9,45.0,658.54
107702,2024/9/8,Amazon,Houston,Final Mile,7,35.0,188.63


# 1. Data Preprocessing  as appropriate to your dataset. 

## (a) cleaning the data

In [4]:
# delete duplicate values
df=df.drop_duplicates()

In [5]:
# check type of data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126255 entries, 0 to 126254
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   WorkDate        126255 non-null  object 
 1   Customer        126255 non-null  object 
 2   Location        126255 non-null  object 
 3   BusinessType    126255 non-null  object 
 4   OrderCount      126255 non-null  object 
 5   NumberOfPieces  126254 non-null  float64
 6   TotalRevenue    126254 non-null  float64
dtypes: float64(2), object(5)
memory usage: 6.7+ MB


In [6]:
# the column of OrderCount should be in int. type, but it is in object type. That shows there is abnormal data.
mask = ~df["OrderCount"].apply(lambda x: str(x).isdigit())
print(df[mask])


    WorkDate  Customer Location BusinessType OrderCount  NumberOfPieces  \
16  2020/1/2  Chipotle  Chicago   Final Mile        XXX           184.0   

    TotalRevenue  
16        2565.8  


In [7]:
# delete abnormal data
df = df.drop(df[mask].index)
df["OrderCount"] = df["OrderCount"].astype(int)

## (b) missing data imputation

In [8]:
# check for missing values
df[df.isna().any(axis=1)]

,WorkDate,Customer,Location,BusinessType,OrderCount,NumberOfPieces,TotalRevenue
20,2020/1/2,Lowes,Sacramento,Final Mile,23,NaN,4729.94
32,2020/1/2,Costco,Detroit,First Mile,7,35.0,NaN


In [9]:
# Fill missing values with the mean
df["NumberOfPieces"] = df["NumberOfPieces"].fillna(df["NumberOfPieces"].mean())
df["TotalRevenue"] = df["TotalRevenue"].fillna(df["TotalRevenue"].mean())

## (c) dealing with outliers

In [10]:
# define a function to remove outliers of each column, using IQR method
def remove_outliers(table,column):
    Q1 = table[column].quantile(0.25)
    Q3 = table[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    condition_delete_outliers=(table[column] >= lower_bound) & (table[column] <= upper_bound)
    table = table[condition_delete_outliers]
    return table

In [11]:
# remove the outliers from the columns of OrderCount, NumberOfPieces,TotalRevenue
df = remove_outliers(df, "OrderCount")
df = remove_outliers(df, "NumberOfPieces")
df = remove_outliers(df, "TotalRevenue")

# 2. EDA & 3. insightful comments related to features

In [13]:
# conduct descriptive satistics
df.describe().round(2)

,OrderCount,NumberOfPieces,TotalRevenue
count,108337.00,108337.00,108337.00
mean,21.17,104.83,1563.18
std,14.93,74.37,1239.53
min,1.00,3.00,26.02
25%,9.00,45.00,606.98
50%,17.00,85.00,1188.04
75%,29.00,145.00,2164.06
max,76.00,345.00,5523.53


In [40]:
# work out each day's total revenue, order count, number of pieces, of which total revenue serves as the dependent variable.
# reorder the index
newdata=df.groupby("WorkDate")[["OrderCount","NumberOfPieces","TotalRevenue"]].sum().reset_index()
newdata.head()

,WorkDate,OrderCount,NumberOfPieces,TotalRevenue
0,2020/1/10,1557,7643.0,117686.27
1,2020/1/11,588,2974.0,53006.51
2,2020/1/12,696,3488.0,61170.18
3,2020/1/13,1369,6577.0,95755.65
4,2020/1/14,1284,6200.0,81527.76


In [42]:
corr_matrix = df[['OrderCount', 'NumberOfPieces', 'TotalRevenue']].corr()
print(corr_matrix)

                OrderCount  NumberOfPieces  TotalRevenue
OrderCount        1.000000        0.973004      0.574847
NumberOfPieces    0.973004        1.000000      0.589955
TotalRevenue      0.574847        0.589955      1.000000


##### Comment: OrderCount and NumberOfPieces are highly correlated, so consider using PCA for dimensionality reduction.

In [41]:
# Use PCA to reduce dimension from 2 to 1
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[['OrderCount','NumberOfPieces']]      # select two variables 
X_scaled = StandardScaler().fit_transform(X)  # Data Standardized to remove the effect of differing scales
pca = PCA(n_components=1)                     # Reduce the original two-dimensional features to one principal component.
X_pca = pca.fit_transform(X_scaled)           # extract PC1

print(pca.explained_variance_ratio_)           # The proportion of total variance explained by the principal component.

[0.98650214]


##### Comment: The proportion of total variance is 0.98650214. Therefore, we have sufficient reason to use PC1.

In [39]:
# combine the PC1 and dependent variable, TotalRevenue
df_pca= pd.DataFrame(X_pca, columns=['PC1'])
df_pca['TotalRevenue'] = newdata['TotalRevenue']
df_pca.head()

,PC1,TotalRevenue
0,1.606802,117686.27
1,-0.462291,53006.51
2,1.132315,61170.18
3,1.227212,95755.65
4,-0.291146,81527.76


# 4. Briefly discuss the question you could potentially explore with the dataset. Back up your reasons using the EDA you have performed with respect your target variable.

"""

I plan to build on the OrderCount and NumberOfPieces to make forecast on the TotalRevenue, using linear regression.
In that case, the OrderCount and NumberOfPieces will act as independent variables and the TotalRevenue as dependent variable.

Through examining correlation between these variables and the variable, I found that the independent variables are highly correlated.  
In linear regression, this can lead to multicollinearity. To avoid this issue, 
I think it is necessary to use dimensionality reduction to eliminate it.


According to the result of PCA, the proportion of total variance explained by the principal component is rather high. Thus, we have the very
reason to use PC1.

Therefore, I combine the PC1 and Total Revenue as the new dataset for futher linear regression analysis.

"""